In [1]:
import numpy as np
import pandas as pd
import sys

In [2]:
# Depth-First Search
# url: https://eddmann.com/posts/depth-first-search-and-breadth-fi

def dfs_paths(graph, start):
    stack = [(start, [start])]
    while stack:
        (vertex, path) = stack.pop()
        for next in graph[vertex] - set(path):
            group_now = graph[vertex]-set(path)
            group_next = graph[next]-set(path)
            if len(group_next) == 0 or (len(group_now) <=1 and len(group_now) == len(group_next)):
                yield path + [next]
            else:
                stack.append((next, path + [next]))

def GetCost(graph, path):
    if len(path) > 1:
        _from = path[0]
        _to = path[1]
        cost = graph[_from][_to]
        for x in path[2:]:
            _from = _to
            _to = x
            cost += graph[_from][_to]
    else:
        cost = 0
    return cost

def searchAllPlaces(grafo, places):
    total_places = len(places)
    total_paths_complete = []
    for place in places:
        list_paths = list(dfs_paths(grafo, place))
        paths_complete = [ x for x in list_paths if len(x)==total_places ] 
        if len(paths_complete) > 0:
            total_paths_complete.append(paths_complete)
    total_paths_complete =np.concatenate(total_paths_complete)
    return total_paths_complete


def getPathsMinimize(paths, distances):
    min_cost = sys.maxsize
    min_path = None
    for path in paths:
        cost = GetCost(distances, path)
        if cost < min_cost:
            min_cost = cost
            min_path = path
    return min_path, min_cost

def loadData(PATH):
    data = pd.read_csv(PATH, sep="=", header=None)
    data = data.rename({0: 'cities', 1: 'km'}, axis='columns')
    
    cities = data['cities'].str.split("to", n = 1, expand = True) 
    data["from"]= cities[0].str.strip()
    data["to"]= cities[1].str.strip()
    data = data.drop(columns="cities") 
    places1 = data['from'].unique()
    places2 = data['to'].unique()
    places_join = np.concatenate([places1, places2])
    places,_ = np.unique(places_join, return_index=True)
    return data, places

def createGraph(data, places, bidirecional=True):
    grafo = {}
    grafo_distances = {}
    data = data.sort_values(by=['from'])

    # Una direccion     
    for _from in places:
        table = data[(data['from']==_from)]
        set_to = []
        grafo_distances[_from] = {}
        for _, _to in table.iterrows():
            set_to.append(_to['to'])
            grafo_distances[_from][_to['to']] = _to['km']
        grafo[_from] = set(set_to)
    
    # Bi-direccional
    if bidirecional:
        data = data.sort_values(by=['to'])
        for _from in places:
            table = data[(data['to']==_from)]
            set_to = []
            if not _from in grafo_distances:
                grafo_distances[_from] = {}

            for _, _to in table.iterrows():
                set_to.append(_to['from'])
                grafo_distances[_from][_to['from']] = _to['km']

            if _from in grafo:
                grafo[_from] = grafo[_from] | set(set_to)
            else:
                grafo[_from] = set(set_to)
            
    return grafo, grafo_distances
    


In [3]:
# MAIN GENERAL
# ------------

PATH_FILE = 'rutas.txt'

# Load Data
# data : Dataframe que contiene la data del archivo TXT
# places : Lista de todos los lugares registrados en el TXT
data, places = loadData(PATH_FILE)

# Generate Graphs
# grafo : contiene las relaciones de los nodos
# grafo_distances : contiene las distancias entre los nodos
grafo, grafo_distances = createGraph(data, places, bidirecional=True)

# Obtenemos todas las rutas completas posibles
all_paths = searchAllPlaces(grafo, places)
# Comparamos las distancias generadas por cada ruta y elegimos la menor
path_min, cost_min = getPathsMinimize(all_paths, grafo_distances)
print(path_min, cost_min)


['Arbre' 'Tambi' 'Snowdin' 'Faerun' 'Straylight' 'Norrath' 'AlphaCentauri'
 'Tristram'] 141
